<a href="https://colab.research.google.com/github/VishalKandala/Reveille-TD/blob/main/Reveille%40TD.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install streamlit

     |████████████████████████████████| 7.4MB 2.7MB/s 
     |████████████████████████████████| 163kB 41.1MB/s 
     |████████████████████████████████| 133kB 42.9MB/s 
     |████████████████████████████████| 112kB 31.7MB/s 
     |████████████████████████████████| 6.7MB 36.4MB/s 
     |████████████████████████████████| 102kB 10.1MB/s 
     |████████████████████████████████| 4.4MB 36.2MB/s 
     |████████████████████████████████| 71kB 8.8MB/s 
     |████████████████████████████████| 71kB 7.5MB/s 
     |████████████████████████████████| 122kB 39.4MB/s 
  Created wheel for blinker: filename=blinker-1.4-cp36-none-any.whl size=13450 sha256=f45969d0ea2a38045916d66fad1fdf4464c99a3f7882275dafca4f526e91ddeb
  Stored in directory: /root/.cache/pip/wheels/92/a0/00/8690a57883956a301d91cf4ec999cc0b258b01e3f548f86e89
  Created wheel for watchdog: filename=watchdog-0.10.3-cp36-none-any.whl size=73873 sha256=d82e91f50d41a90bf9776fc786b84d9b5bbfa9b6890244e50dfbe4dc8c60413f
  Stored in directory: /root/.c

In [2]:
!pip install pyngrok

  Created wheel for pyngrok: filename=pyngrok-4.2.2-cp36-none-any.whl size=18025 sha256=f7c4c06764067349ffd93111296402cf30ea5c5f4d6ae11fbdd2eb57aac7c311
  Stored in directory: /root/.cache/pip/wheels/9c/d3/b5/502fbef4b2782a6142bc00c62ccde76ee04c9148fb43b1b2cd
Successfully built pyngrok


In [3]:
!cd /content/drive/My\ Drive/Colab\ Notebooks/

/bin/bash: line 0: cd: /content/drive/My Drive/Colab Notebooks/: No such file or directory


In [4]:
%%writefile style.css
body {
    color: black;
    background-color: #4F8BF9;
    #background-color: transparent;
    #background-image: linear-gradient(to bottom, black,black),url("https://drive.google.com/uc?export=download&id=1bG_rkbQq_Oh3935tkCISLg3Dl_h8GwGQ");  
    #background-size: cover;
    #opacity:0.5;

}

sidebar .sidebar-content {
    background-image: linear-gradient('#2e7bcf','#2e7bcf');
    color: black;
    #background-color: #FFDF00;

.stButton>button {
    color: black;
    border-radius: 50%;
    height: 3em;
    width: 3em;
}

.stTextInput>div>div>input {
    color: #4F8BF9;
}

Writing style.css


In [5]:
%%writefile Teammaker.py
import pandas as pd
import numpy as np
import streamlit as st
import string
import Home
#######################
apps=Home.dset
def remove_punc(text):
    b="".join([char for char in text if char not in '?:\'/\/.*[\'\]*'])
    return b

def disp_prof(id):
    data=apps[['school','majors','classification','datascience_experience','num_hackathons_attended','technology_experience']]
    data=pd.DataFrame(data.values)
    s=apps.index[apps['userid']==id]
    exp=st.beta_expander(id)
    with exp:
      st.write('Education:')
      scl=str(data[0].values[s])
      scl=remove_punc(scl)
      st.write('School: ',scl)
      cl=str(data[2].values[s])
      cl=remove_punc(cl)
      if (cl=='Ma'):
        st.write('Classification: Masters')
      elif (cl=='Jr'):
        st.write('Classification: Junior')
      elif cl=='Sr':
        st.write('Classification: Senior')
      elif cl=='So':
        st.write('Classification: Sophomore')
      elif cl=='Fr':
        st.write('Classification: Freshman')
      elif cl=='Phd':
        st.write('Classification: Doctoral Student')
      elif cl=='O':
        st.write('Classification: Other')
      a=str(data[1].values[s]).split(',')
      for i in range(len(data[1])):
        a=str(data[1].values[i]).split(',')
        major=remove_punc(a[0])
      st.write('Major: ',major)
      st.write('Experience:')
      datexp=str(data[3].values[s])
      datexp=remove_punc(datexp)
      st.write('Datascience Experience (yrs): ',datexp)
      hacks=str(data[4].values[s])
      hacks=remove_punc(hacks)
      st.write('Hackathons Attended: ',hacks)

def similar_people(id):
    data=apps[['school','majors','classification','datascience_experience','num_hackathons_attended']]
    a=data['majors'][0].split(',')
    for i in range(len(data['majors'])):
        a=data['majors'][i].split(',')
        data['majors'][i]=remove_punc(a[0])
    from sklearn.preprocessing import LabelEncoder
    labelencoder = LabelEncoder()
    data['school'] = labelencoder.fit_transform(data['school'])
    data['majors'] = labelencoder.fit_transform(data['majors'])
    data['classification'] = labelencoder.fit_transform(data['classification'])
    data['num_hackathons_attended'] = labelencoder.fit_transform(data['num_hackathons_attended'])
    from sklearn.preprocessing import OneHotEncoder
    oh=OneHotEncoder(sparse=False)
    data=pd.DataFrame(oh.fit_transform(data))
    from sklearn.metrics.pairwise import cosine_similarity
    q=cosine_similarity(data)
    s=apps.index[apps['userid']==id].tolist()
    apps['sim']=q[s].T
    w=np.where(q[s]>0.7)
    if len(w[1])<10:
        w=np.where(q[s]>0.5)
    out=apps.iloc[w[1]]
    out=pd.DataFrame(out.values,columns=out.columns)
    out=out.sort_values(by=['sim'],ascending=False)
    iskill=[]
    
    for i in out['technology_experience'][0].split(','):
        iskill.append(remove_punc(i))
    g=len(set(iskill))
    ind=0
    f=pd.DataFrame(columns=out.columns)
    for k in range(5):
        for i in range(len(out['technology_experience'])):
            tskill=[]
            for j in out['technology_experience'][i].split(','):
                tskill.append(remove_punc(j))
            t=iskill+tskill
            s=len(set(t))
            if s>g:
                g=s
                ind=i
        f=f.append(out.iloc[ind])
        out.drop([0],inplace=True)
        out=pd.DataFrame(out.values,columns=out.columns)
        f=pd.DataFrame(f.values,columns=f.columns)
        #f.drop(f.columns[[1,2,3,4,5,6,7,8,9,10,11,12,13]], axis=1,inplace=True)
    return f
###########################################
def app():
  st.header('Your Dream Team')
  st.write('Below are our Top 5 Recommendations for you, to consider as your potential teammates, go ahead connect with them on discord and other platforms!')
  uid=Home.id
  teammates=similar_people(uid)
  for i in range(len(teammates)):
    id=teammates['userid'][i]#pd.Series.to_string(teammates['userid'][i])
    disp_prof(id)  


Writing Teammaker.py


In [6]:
%%writefile WorkshopRec.py
import pandas as pd
import numpy as np
import streamlit as st
import string
import Home
#######################
apps=Home.dset
apps=apps.set_index('userid')
def remove_punc(text):
    b="".join([char for char in text if char not in '?:\'/\/.*[\'\]*'])
    return b
def workshop(id):
    data=apps[['datascience_experience','num_hackathons_attended']]
    data['expertise']='Beginner'
    for i in range(len(data['datascience_experience'])):
        if data['datascience_experience'][i]>=4 and data['num_hackathons_attended'][i] in ['10+', '8-10']:
            data['expertise'][i]='Expert'
        elif data['datascience_experience'][i]>2 or data['num_hackathons_attended'][i] in ['8-10','4-7','1-3']:
            data['expertise'][i]='Intermediate'
        else:
            data['expertise'][i]='Beginner'


    apps['expertise']=data['expertise']
    wrk=pd.read_csv('https://drive.google.com/uc?id=10MngpIZoAGgwAk_sxoORj7WPYs74nz5Y')
    wrk=wrk.iloc[:13]
    if apps['expertise'][id]=='Beginner':
        return wrk[wrk['difficulty']==0]
    elif apps['expertise'][id]=='Intermediate':
        return wrk[wrk['difficulty']==1]
    else:
        return wrk[(wrk['difficulty']==3)|(wrk['difficulty']==2)]

def disp_workshop(wi):
  exp=st.beta_expander(wi[1])
  with exp:
    #desc=pd.Series.to_string(wi[3])
    st.write(wi[3])
    #track=pd.Series.to_string(wi[4])
    st.write('Track:',wi[4])

def app():
  st.header('Need help?')
  st.write('Do not fret for we have got you covered, check out these workshops we carefully selected for you based on your proficiency as well as interests!')
  uid=Home.id
  workshop_rec=workshop(uid)
  workshop_rec=pd.DataFrame(workshop_rec.values)
  #st.write(workshop_rec)
  for i in range(len(workshop_rec)):
    #pd.Series.to_string(teammates['userid'][i])
    disp_workshop(workshop_rec.loc[i])


Writing WorkshopRec.py


In [7]:
%%writefile Home.py
import streamlit as st
import pandas as pd
def expertise(dset):    
    data=dset[['datascience_experience','num_hackathons_attended']]
    data['expertise']='Beginner'
    for i in range(len(data['datascience_experience'])):
        if data['datascience_experience'][i]>=4 and data['num_hackathons_attended'][i] in ['10+', '8-10']:
            data['expertise'][i]='Expert'
        elif data['datascience_experience'][i]>2 or data['num_hackathons_attended'][i] in ['8-10','4-7','1-3']:
            data['expertise'][i]='Intermediate'
        else:
            data['expertise'][i]='Beginner'
    dset['expertise']=data['expertise']
    return dset
            

    apps['expertise']=data['expertise']
def app():
  st.header('HOWDY! Welcome to TAMU Datathon')
  st.write('This is your one stop shop for all things Datathon, from team mate recommendations to workshop recommendations and great articles.')
  st.write('Here, you can also find a very helpfull companion in our chatbot Reville!')
  st.write('In the sidebar you will find links that will take you to our Team Builder, as well as the workshop recommender,while Reville can be accessed on the bottom right corner.')
id='95030aef3e970680d882f1527f549d9e' # Hard coding User ID for n
dset1 = pd.read_csv('https://drive.google.com/uc?id=1mPK8_AasPMdqy3D9D0kxjjKcIXmhhcQo')
dset=expertise(dset1)


Writing Home.py


In [8]:
%%writefile Stats.py
import streamlit as st
from math import pi
import Home
import matplotlib.pyplot as plt
dats=Home.dset
import pandas as pd
from bokeh.io import output_file, show
from bokeh.palettes import Category20c
from bokeh.plotting import figure
from bokeh.transform import cumsum
import plotly.express as px

datset=dats[['school','majors','classification','expertise','num_hackathons_attended','relavent_industries']]
def remove_punc(text):
    b="".join([char for char in text if char not in '?:\'/\/.*[\'\]*\"\_'])
    return b
def app():
  st.header("Analytics")
  options=['school','expertise','Industry']
  par=st.selectbox('Select the Parameter to Display Data',options)
  if par=='school':
    df=datset['school'].value_counts()[:20]
  elif par=='expertise':
    df=datset['expertise'].value_counts()[:20]
  elif par=='Industry':
    df=datset['relavent_industries'].value_counts()[:20]
  x=df.to_dict()
  df1=pd.Series(x).reset_index(name='value').rename(columns={'index':'country'})
  df1['angle'] = df1['value']/df1['value'].sum() * 2*pi
  df1['color'] = Category20c[len(x)]
  p = figure(plot_height=350, title="Pie Chart", toolbar_location=None,
    tools="hover", tooltips="@country: @value", x_range=(-0.5, 1.0))
  p.wedge(x=0, y=1, radius=0.4,
    start_angle=cumsum('angle', include_zero=True), end_angle=cumsum('angle'),
    line_color="white", fill_color='color', legend_field='country', source=df1)
  p.axis.axis_label=None
  p.axis.visible=False
  p.grid.grid_line_color = None
  st.bokeh_chart(p)

Writing Stats.py


In [9]:
!pip install nltk

In [10]:
%%writefile Chatbot.py
import nltk
import streamlit as st
from keras.models import Sequential
from keras.layers import Dense
from nltk.stem.lancaster import LancasterStemmer
stemmer=LancasterStemmer()
nltk.download('punkt') 
import numpy as np
import tensorflow
#import tflearn
import random
import json
import pickle
import urllib
import os
url='https://drive.google.com/uc?export=download&id=1vM6shVEnKPiwpV0tzJatRGZD5XN7HCFX'
response=urllib.request.urlopen(url)
data=json.loads(response.read())

try:
    with open('data.pickle','rb') as f:
        words,labels,training,output=pickle.load(f)

except:

    words=[]
    labels=[]
    docs_x=[]
    docs_y=[]

    for intent in data['intents']:
        for pattern in intent['patterns']:
            wrds=nltk.word_tokenize(pattern)
            words.extend(wrds)
            docs_x.append(wrds)
            docs_y.append(intent['tag'])

            if intent['tag'] not in labels:
                labels.append(intent['tag'])
            
    words=[stemmer.stem(w.lower()) for w in words if w !='?']
    words=sorted(list(set(words)))

    labels=sorted(labels)

    training=[]
    output=[]

    out_empty=[0 for _ in range(len(labels))]
    for x,doc in enumerate(docs_x):
        bag=[]
        wrds=[stemmer.stem(w) for w in doc]
        for w in words:
            if w in wrds:
                bag.append(1)
            else:
                bag.append(0)
        output_row=out_empty[:]
        output_row[labels.index(docs_y[x])]=1

        training.append(bag)
        output.append(output_row)

    training=np.array(training)
    output=np.array(output)
    with open('data.pickle','wb') as f:
        pickle.dump((words,labels,training,output),f)

model = Sequential()
model.add(Dense(len(training[0]),input_dim=len(training[0]), kernel_initializer='normal', activation='relu'))
model.add(Dense(32))
model.add(Dense(32))
model.add(Dense(len(output[0]), kernel_initializer='normal'))
model.compile(loss='mean_squared_error', optimizer='adam')
try:
    model.load_weights('model.hdf5')
except:
    model.fit(training,output,epochs=1000, batch_size=8)
    print('done')
model.save('model.hdf5')

def bag_of_words(s,words):
    bag=[0 for _ in range(len(words))]

    s_words=nltk.word_tokenize(s)
    s_words=[stemmer.stem(word.lower()) for word in s_words]

    for se in s_words:
        for i,w in enumerate(words):
            if w==se:
                bag[i]=1
    return np.array(bag)
hist=[]
def chat():
    inp=st.text_input("you:")
    #hist.append(inp)

        #if inp.lower()=="quit":
        #    exit(0)
    results=model.predict(np.expand_dims(np.array(bag_of_words(inp,words)),axis=0))
    results_index=np.argmax(results)
    tag=labels[results_index]    
    for tg in data['intents']:
        if tg['tag']==tag:
            responses=tg['responses']
    out=random.choice(responses)
    st.write(out)

def app():
    st.header('Ask Me Anything!')
    st.write('Hi, I am Reveille, I am here to help you with any queries you may have regarding workshops,Sponsors and much more!')
    chat()


Writing Chatbot.py


In [11]:
%%writefile webapp.py
import Teammaker
import Home
import Stats
import WorkshopRec
import Chatbot
import streamlit as st

def local_css(file_name):
    with open(file_name) as f:
        st.markdown(f'<style>{f.read()}</style>', unsafe_allow_html=True)

def remote_css(url):
    st.markdown(f'<link href="{url}" rel="stylesheet">', unsafe_allow_html=True)    

def icon(icon_name):
    st.markdown(f'<i class="material-icons">{icon_name}</i>', unsafe_allow_html=True)
local_css('style.css')
remote_css('https://fonts.googleapis.com/icon?family=Material+Icons')

PAGES = {
    "Home":Home,
    "Teams":Teammaker,
    "Workshops ":WorkshopRec,
    "Analytics": Stats,
    "Reville: The Bot":Chatbot
}

#st.markdown(page_bg_img, unsafe_allow_html=True)
st.sidebar.title('Navigation')
selection = st.sidebar.radio("Go to", list(PAGES.keys()))
page = PAGES[selection]
page.app()

Writing webapp.py


In [12]:
!ls

Chatbot.py  sample_data  style.css     webapp.py
Home.py     Stats.py	 Teammaker.py  WorkshopRec.py


In [13]:
!streamlit run webapp.py &>/dev/null&

In [18]:
from pyngrok import ngrok
# Setup a tunnel to the streamlit port 8501
public_url = ngrok.connect(port='8501')
print(public_url)

http://1308c010a82a.ngrok.io


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: DeprecationWarning: Support for "return_ngrok_tunnel" as "False" is deprecated and will be removed in 5.0.0, when this method will return a NgrokTunnel instead of a str
  This is separate from the ipykernel package so we can avoid doing imports until


To Kill the tunnel and the site. Use the following Codeblock.

In [15]:
!pgrep streamlit

217


In [16]:
!kill 951 992 2877

/bin/bash: line 0: kill: (951) - No such process
/bin/bash: line 0: kill: (992) - No such process
/bin/bash: line 0: kill: (2877) - No such process


In [17]:
ngrok.kill()